## Mission to Mars

In [6]:
# Import dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser


In [7]:
# Choose the executable path to driver 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [8]:
# Open desired site in browser with splinter
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [9]:
# HTML Object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [35]:
# Retrieve the latest element that contains news title and news_paragraph
title_results = soup.find_all('div', class_='content_title')
p_results = soup.find_all('div', class_='article_teaser_body')
news_date = soup.find_all('div', class_='list_date').text

# Extract first title and paragraph, and assign to variables
news_title = title_results[0].text
news_p = p_results[0].text

print(f'Article date: {news_date}')
print(f'Title: {news_title}')
print(f'Abstract: {news_p}')

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

### JPL Mars Space Images - Featured Image

### Mars Facts

### Mars Hemispheres